In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f'''
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_db_inventory.a_dc_store_nonfresh_daily AS
--PARTITION BY calday CLUSTER BY store_id, product_id AS
-- DANH MỤC SKULINE LÀ CÁC SKU QUA LUỒNG DC, LẤY THEO BẢNG HIST
WITH
store_sku AS (
  SELECT
    a.calday,
    a.store_id,
    a.product_id,
    a.dc_id,
    b.business_unit,
    CASE
      WHEN b.business_unit = '1500' THEN 'Supermarket'
      WHEN IFNULL(b.concept,'') IN ('Rural','WiN') THEN b.concept
      ELSE 'Urban'
    END concept,
    IFNULL(a.external_report_mdq,0) external_report_mdq,
    IFNULL(a.internal_report_mdq,0) internal_report_mdq,
    IFNULL(a.old_mdq,0) old_mdq,
    a.mdq_age,
    IFNULL(IFNULL(
      a.assortment_type,
      CASE
        WHEN a.calday <'2024-04-02' THEN asm_1.core_range
        WHEN a.calday >='2024-04-02' AND b.business_unit = '1500' THEN asm_3.core_range
        WHEN a.calday >='2024-04-02' AND b.business_unit = '2000' THEN asm_2.core_range
        ELSE 'NA'
      END
    ),'NA') assortment_type,
    IFNULL(a.is_problem_article,'NO') is_problem_article
  FROM
   {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_mdq_store_dc_sku_daily_his a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b USING (store_id)
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_core_range_assortment_his asm_1 ON
      a.calday = asm_1.calday AND
      a.product_id = asm_1.product_id AND
      b.business_unit = asm_1.business_unit AND
      b.city = asm_1.city
    LEFT JOIN {catalog_name}.udp_wcm_gold_dev.d_core_range_assortment_wmp_his asm_2 ON
      a.calday = asm_2.calday AND
      a.product_id = asm_2.product_id AND
      b.business_unit = asm_2.business_unit AND
      b.city = asm_2.city
    LEFT JOIN {catalog_name}.udp_wcm_gold_dev.d_core_range_assortment_wmt_his asm_3 ON
      a.calday = asm_3.calday AND
      a.product_id = asm_3.product_id AND
      a.store_id = asm_3.store_id
  WHERE
    dc_id IS NOT NULL AND
    b.business_unit IN ('2000','1500') AND
    mch2_id NOT IN ('101') AND
    a.calday BETWEEN date_sub(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 61) AND DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS),1)
  --báo cáo hiển thị 6 tuần gần nhất tính cả tuần hiện tại
),
min_stock_temp AS (
  SELECT
    a.* ,
    internal_report_mdq mdq,
    CASE WHEN c.approval_status = 'YES' THEN 'APPROVED MDQ_POG'
    WHEN c.approval_status = 'IN REVIEW' THEN 'IN REVIEW MDQ_POG'
    ELSE 'OLD_MDQ' END mdq_type,
    IFNULL(d.rr,0) rr
  FROM
    store_sku a
    LEFT JOIN (
      SELECT
        calday,
        store_id,
        product_id,
        SUM(sell_rate) rr
      FROM {catalog_name}.udp_wcm_gold_vcm_dmt.f_corrected_sellrate
      WHERE calday BETWEEN DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)- 61 AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)-1
      GROUP BY 1,2,3
    ) d ON a.calday = d.calday AND a.store_id = d.store_id AND a.product_id = d.product_id
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_pog_store_sku c ON a.product_id = c.product_id AND a.store_id = c.store_id
),
min_stock AS (
  SELECT
    a.*,
    GREATEST(rr,mdq) min_stock
  FROM min_stock_temp a
),
po AS (
  SELECT
    a.calday,
    a.store_id,
    a.product_id,
    SUM(DATE_DIFF(a.gr_date, a.creation_date)) total_leadtime,
    COUNT(DISTINCT a.po_id) count_po_gr
  FROM
    {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_item a
    INNER JOIN store_sku b ON a.store_id = b.store_id AND a.product_id = b.product_id
  WHERE
    gr_date >= creation_date
    AND a.calday >= DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 121) AND a.calday < DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
  GROUP BY 1,2,3
),
po_pending AS (
  SELECT
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)-1 calday,
    a.*
  FROM (
    SELECT
      a.store_id,
      a.product_id,
      IFNULL(SUM(a.po_qty * c.numerator/c.denomintr), 0) po_pending_qty,
      IFNULL(SUM(a.po_value), 0) po_pending_val
    FROM
      {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_item a
      INNER JOIN (SELECT DISTINCT store_id,product_id FROM store_sku) b ON a.store_id = b.store_id AND a.product_id = b.product_id
      LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_unit_conv c ON a.product_id = c.material AND a.po_unit = c.mat_unit
    WHERE
      a.creation_date >= DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 21) AND a.creation_date < DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
      AND (a.compl_del <> 'X' OR a.compl_del IS NULL)
      AND (a.po_gr_date IS NULL)
      AND (a.gr_date IS NULL)
      AND (a.webportal_status NOT IN ('Đã hủy', 'Đã giao hàng', 'Quá hạn') OR a.webportal_status IS NULL)
    GROUP BY 1,2
  ) a
),
base_sale_qty_stock_data_daily AS (
  SELECT
    a.calday,
    a.store_id,
    a.product_id,
    IFNULL(b.revenue.base_sale_qty, 0) base_sale_qty,
    IFNULL(IF(business_unit = '2000',revenue.rev_amt_pos,revenue.rev_amt),0) rev_amt,
    IFNULL(b.stock.shrinkage_val,0) shrinkage_val,
    IFNULL(b.revenue.cogs_pos,0) cogs,
    IFNULL(b.stock.closing_stock_quantity_sloc_1000,0) closing_stock_qty,
    IFNULL(b.stock.closing_stock_value,0) closing_stock_val
  FROM
    min_stock a
    LEFT JOIN (
      SELECT * 
      FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_store_sku_daily
      WHERE calday BETWEEN DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 121 AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1
    ) b ON a.calday = b.calday AND a.store_id = b.store_id AND a.product_id = b.product_id
),
base_sale_qty_data_30d AS (
  SELECT
    *,
    SUM(base_sale_qty) OVER (PARTITION BY store_id,product_id ORDER BY calday ASC ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) base_sale_qty_30d,
    SUM(cogs) OVER (PARTITION BY store_id,product_id ORDER BY calday ASC ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) cogs_30d
  FROM base_sale_qty_stock_data_daily
),
cogs_val_30d AS (
  SELECT 
    calday,
    store_id, 
    product_id, 
    SUM(cogs_30d) cogs_30d
  FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_cogs_90d
  WHERE calday BETWEEN DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)- 62 AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)- 1
  GROUP BY 1,2,3
),
main AS (
  SELECT
    a.calday,
    c.isoweek,
    a.store_id,
    a.product_id,
    a.mdq_type,
    a.mdq,
    a.external_report_mdq,
    a.internal_report_mdq,
    a.old_mdq,
    a.min_stock,
    a.rr,
    a.dc_id,
    a.assortment_type,
    a.mdq_age,
    a.is_problem_article,
    IF(k.store_id IS NOT NULL,"YES","NO") block_high_stock,
    IF(f.store_id IS NOT NULL,"YES","NO") is_hibeauty,
    IF(g.store_id IS NOT NULL,"YES",'NO') is_hero,
    h.pilot_type,
    IF(min_stock > 0, 1, 0) dr_count_internal,
    IF(min_stock > 0 AND IFNULL(d.closing_stock_qty, 0) >= min_stock,1,0) dr_internal,
    IF(min_stock > 0 AND (IFNULL(d.closing_stock_qty, 0) + IFNULL(b1.po_pending_qty, 0))>=min_stock, 1, 0) adj_dr_internal,
    IF (a.external_report_mdq > 0 AND mdq_age >= 14, 1, 0) dr_count_external,
    IF(IFNULL(d.closing_stock_qty, 0) >= a.external_report_mdq AND external_report_mdq > 0 AND mdq_age >= 14, 1, 0) dr_external,
    CASE
      WHEN (IFNULL(d.closing_stock_qty, 0) + IFNULL(b1.po_pending_qty, 0)) >= external_report_mdq AND mdq_age>=14 AND external_report_mdq>0 THEN 1
      ELSE 0
    END adj_dr_external,
    IFNULL(d1.rev_amt,0) m_rev_amt,
    IFNULL(d1.shrinkage_val, 0) shrinkage_val,
    IFNULL(d1.base_sale_qty, 0) cogs_qty,
    IFNULL(d.closing_stock_qty, 0) closing_stock_qty,
    IFNULL(d.closing_stock_val, 0) closing_stock_val,
    IFNULL(d.cogs,0) AS cogs,
    IFNULL(e.base_sale_qty_30d, 0) cogsq_30d,
    IFNULL(e1.cogs_30d, 0) cogsval_30d,
    IFNULL(b.total_leadtime, 0) total_leadtime,
    IFNULL(b.count_po_gr, 0) count_po_gr,
    IFNULL(b1.po_pending_qty, 0) po_pending_qty,
    IFNULL(b1.po_pending_val, 0) po_pending_val
  FROM 
    min_stock a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st USING (store_id)
    LEFT JOIN po b ON a.calday = b.calday AND a.product_id = b.product_id AND a.store_id = b.store_id
    LEFT JOIN po_pending b1 ON a.calday = b1.calday AND a.product_id = b1.product_id AND a.store_id = b1.store_id
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_time c ON a.calday = c.calday
    LEFT JOIN base_sale_qty_stock_data_daily d ON a.product_id = d.product_id AND a.store_id = d.store_id AND a.calday = d.calday
    LEFT JOIN base_sale_qty_stock_data_daily d1 ON a.product_id = d1.product_id AND a.store_id = d1.store_id AND a.calday = d1.calday
    LEFT JOIN base_sale_qty_data_30d e ON a.product_id = e.product_id AND a.store_id = e.store_id AND a.calday = e.calday
    LEFT JOIN cogs_val_30d e1 ON a.product_id = e1.product_id AND a.store_id = e1.store_id AND a.calday = e1.calday
    LEFT JOIN (SELECT DISTINCT product_id, store_id FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_hibeauty) f ON a.store_id = f.store_id AND a.product_id = f.product_id
    LEFT JOIN (SELECT DISTINCT product_id, store_id FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_hero) g ON a.store_id = g.store_id AND a.product_id = g.product_id
    LEFT JOIN  {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_urban_pilot h ON a.store_id = h.store_id
    LEFT JOIN (SELECT * FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_block_by_store_sku_current_his WHERE reason LIKE "High stock%") k ON
      a.calday = k.calday + 1 AND
      a.product_id = k.product_id AND
      a.store_id = k.store_id
)
SELECT a.* FROM main a
''')